In [25]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import numpy as np

In [9]:
import tarfile
from torchvision.datasets.utils import download_url

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score

In [10]:
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url, '.')

  0%|          | 0/135107811 [00:00<?, ?it/s]

In [11]:
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./img_data')

In [13]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = ImageFolder('./img_data/cifar10'+'/train', transform=transform)


In [66]:
X = torch.tensor([np.array(dataset[i][0]) for i in range(0, len(dataset))], dtype=torch.float32)

In [56]:
Y = torch.tensor([dataset[i][1] for i in range(0, len(dataset))], dtype=torch.long)

In [34]:
import torch.nn as nn
import torch.nn.functional as F


class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # 배치를 제외한 모든 차원을 평탄화(flatten)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [74]:
USE_CUDA = torch.cuda.is_available()

DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

device(type='cpu')

In [76]:
import torch.optim as optim
import gc
from torch.utils.data import DataLoader, TensorDataset

loo = LeaveOneOut()
epochs = 10

cv_result = []

for train_index, test_index in loo.split(X):

    model = CNN().to(DEVICE)
    
    # 손실 함수와 옵티마이저 정의
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    # 훈련 및 검증 데이터 분할
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    # DataLoader를 사용하여 데이터 로드
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
    
    # 모델 훈련
    for epoch in range(epochs):
        running_loss = 0.0
        for i,(inputs, labels) in enumerate(train_loader):
            inputs , labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 1000 == 0:   
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 1000:.3f}')
                running_loss = 0.0

    # # 모델 평가
    with torch.no_grad():
        model.eval()
        X_test = X_test.to(DEVICE)
        test_output = model(X_test)
        predicted = torch.argmax(test_output, dim=1)
        accuracy = (predicted == y_test).sum().item() / y_test.size(0)
        cv_result.append(accuracy)
        print(f"Test Accuracy: {accuracy * 100:.2f}%")

    del model
    gc.collect()

[1,     1] loss: 0.002
[1,  1001] loss: 2.291
[1,  2001] loss: 2.059
[1,  3001] loss: 1.834
[1,  4001] loss: 1.688
[2,     1] loss: 0.002
[2,  1001] loss: 1.537
[2,  2001] loss: 1.498
[2,  3001] loss: 1.434
[2,  4001] loss: 1.422
Test Accuracy: 0.00%
[1,     1] loss: 0.002
[1,  1001] loss: 2.287
[1,  2001] loss: 2.085
[1,  3001] loss: 1.868
[1,  4001] loss: 1.726
[2,     1] loss: 0.002
[2,  1001] loss: 1.553
[2,  2001] loss: 1.510
[2,  3001] loss: 1.468
[2,  4001] loss: 1.412
Test Accuracy: 0.00%
[1,     1] loss: 0.002
[1,  1001] loss: 2.247
[1,  2001] loss: 2.064
[1,  3001] loss: 1.844
[1,  4001] loss: 1.706
[2,     1] loss: 0.002
[2,  1001] loss: 1.543
[2,  2001] loss: 1.503
[2,  3001] loss: 1.451
[2,  4001] loss: 1.421
Test Accuracy: 0.00%
[1,     1] loss: 0.002
[1,  1001] loss: 2.289
[1,  2001] loss: 2.052
[1,  3001] loss: 1.844
[1,  4001] loss: 1.697
[2,     1] loss: 0.002


KeyboardInterrupt: 

In [ ]:
# LOOCV 평균 점수 계산
mean_score = np.mean(cv_result)
print(f"Mean Test Accuracy: {mean_score * 100:.2f}%")